In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as mtp
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import math
import nltk

In [2]:
a=pd.read_csv("C:\\Users\\sowmy\\Downloads\\Paper2\\Sentiment.csv")

In [3]:
df=pd.DataFrame(a)
df

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,...,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,1,No candidate mentioned,1.0000,yes,1.0000,Neutral,0.6578,None of the above,1.0000,NaN,...,NaN,5,NaN,NaN,RT @NancyLeeGrahn: How did everyone feel about...,NaN,2015-08-07 09:54:46 -0700,629697200650592256,NaN,Quito
1,2,Scott Walker,1.0000,yes,1.0000,Positive,0.6333,None of the above,1.0000,NaN,...,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN
2,3,No candidate mentioned,1.0000,yes,1.0000,Neutral,0.6629,None of the above,0.6629,NaN,...,NaN,27,NaN,NaN,RT @TJMShow: No mention of Tamir Rice and the ...,NaN,2015-08-07 09:54:46 -0700,629697199312482304,NaN,NaN
3,4,No candidate mentioned,1.0000,yes,1.0000,Positive,1.0000,None of the above,0.7039,NaN,...,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)
4,5,Donald Trump,1.0000,yes,1.0000,Positive,0.7045,None of the above,1.0000,NaN,...,NaN,156,NaN,NaN,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,NaN,2015-08-07 09:54:45 -0700,629697196967903232,NaN,Arizona
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13866,13867,No candidate mentioned,1.0000,yes,1.0000,Negative,0.7991,Abortion,0.6014,No candidate mentioned,...,yes,7,Negative,Abortion\nWomen's Issues (not abortion though),RT @cappy_yarbrough: Love to see men who will ...,NaN,2015-08-07 09:29:43 -0700,629690895479250944,Como,NaN
13867,13868,Mike Huckabee,0.9611,yes,1.0000,Positive,0.7302,None of the above,0.9229,Mike Huckabee,...,yes,1,NaN,NaN,RT @georgehenryw: Who thought Huckabee exceede...,NaN,2015-08-07 09:25:02 -0700,629689719056568320,USA,NaN
13868,13869,Ted Cruz,1.0000,yes,1.0000,Positive,0.8051,None of the above,0.9647,Ted Cruz,...,yes,67,Positive\nNeutral,NaN,"RT @Lrihendry: #TedCruz As President, I will a...",NaN,2015-08-07 07:19:18 -0700,629658075784282112,NaN,NaN
13869,13870,Donald Trump,1.0000,yes,1.0000,Negative,1.0000,Women's Issues (not abortion though),0.9202,Donald Trump,...,yes,149,NaN,Women's Issues (not abortion though),RT @JRehling: #GOPDebate Donald Trump says tha...,NaN,2015-08-07 09:54:04 -0700,629697023663546368,NaN,NaN


In [4]:
df.columns

Index(['id', 'candidate', 'candidate_confidence', 'relevant_yn',
       'relevant_yn_confidence', 'sentiment', 'sentiment_confidence',
       'subject_matter', 'subject_matter_confidence', 'candidate_gold', 'name',
       'relevant_yn_gold', 'retweet_count', 'sentiment_gold',
       'subject_matter_gold', 'text', 'tweet_coord', 'tweet_created',
       'tweet_id', 'tweet_location', 'user_timezone'],
      dtype='object')

In [5]:
df=df[["text","sentiment"]]
df.shape

(13871, 2)

In [6]:
df.duplicated().sum()

3304

In [7]:
df_dup=df[df.duplicated(keep="first")]
df_dup

,text,sentiment
7887,"RT @goldengateblond: ""It's only by the blood o...",Negative
7888,RT @rikardorichards: #DonaldTrump and #JebBush...,Neutral
7889,#GOPDebate Never lose focus on the truly impor...,Neutral
7890,RT @RedheadAndRight: Meet the new darling of t...,Neutral
7891,RT @ChrisJZullo: Last night they could have ha...,Negative
...,...,...
13865,RT @RWSurferGirl: Fox is cherry picking the ca...,Negative
13866,RT @cappy_yarbrough: Love to see men who will ...,Negative
13868,"RT @Lrihendry: #TedCruz As President, I will a...",Positive
13869,RT @JRehling: #GOPDebate Donald Trump says tha...,Negative


In [8]:
df_dups=df.drop_duplicates()
df_dups

,text,sentiment
0,RT @NancyLeeGrahn: How did everyone feel about...,Neutral
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
2,RT @TJMShow: No mention of Tamir Rice and the ...,Neutral
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive
...,...,...
13846,Me watching the GOP Debates. #gopdebates https...,Neutral
13847,RT @mjtbaum: GOD is making an appearance at th...,Positive
13857,"This is why I don't watch Fox News, they're al...",Negative
13859,"Best line of #GOPDebate was ""Immigration witho...",Positive


In [9]:
df=df_dups
df

,text,sentiment
0,RT @NancyLeeGrahn: How did everyone feel about...,Neutral
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
2,RT @TJMShow: No mention of Tamir Rice and the ...,Neutral
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive
...,...,...
13846,Me watching the GOP Debates. #gopdebates https...,Neutral
13847,RT @mjtbaum: GOD is making an appearance at th...,Positive
13857,"This is why I don't watch Fox News, they're al...",Negative
13859,"Best line of #GOPDebate was ""Immigration witho...",Positive


In [10]:
df.duplicated().sum()

0

DROPPING NEUTRALS


In [11]:
df.drop(df.index[df['sentiment'] == "Neutral"], inplace=True)

# Print the total number of positive and negative sentiments.


In [39]:
df["sentiment"].value_counts()

Negative    6129
Positive    1709
Name: sentiment, dtype: int64

# categorical values to numeric using the factorize() method 

In [20]:
sentiment_label = df.sentiment.factorize()
sentiment_label

(array([0, 0, 0, ..., 1, 0, 0], dtype=int64),
 Index(['Positive', 'Negative'], dtype='object'))

# REMOVE STOPWORDS

In [18]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sowmy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
tweet = df.text.values

## tokenizing

In [23]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(tweet)

## text_to_sequence() method

In [24]:
encoded_docs = tokenizer.texts_to_sequences(tweet)

In [25]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_sequence = pad_sequences(encoded_docs, maxlen=200)

# Build a sequential LSTM model to predict positive and negative sentiments.


In [42]:
reviews = df['text'].values
labels = df['sentiment'].values
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)

In [43]:
train_sentences, test_sentences, train_labels, test_labels = train_test_split(reviews, encoded_labels, stratify = encoded_labels)


In [50]:
vocab_size = 3000 
oov_tok = ''
embedding_dim = 100
max_length = 200 
padding_type='post'
trunc_type='post'

In [51]:
# tokenize sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

In [52]:
# convert train dataset to sequence and pad sequences
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, padding='post', maxlen=max_length)

In [53]:
# convert Test dataset to sequence and pad sequences
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_length)

In [45]:
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
# compile model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# model summary
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 100)          300000    
                                                                 
 bidirectional (Bidirectiona  (None, 128)              84480     
 l)                                                              
                                                                 
 dense_2 (Dense)             (None, 24)                3096      
                                                                 
 dense_3 (Dense)             (None, 1)                 25        
                                                                 
Total params: 387,601
Trainable params: 387,601
Non-trainable params: 0
_________________________________________________________________


In [46]:
num_epochs = 5
history = model.fit(train_padded, train_labels, 
                    epochs=num_epochs, verbose=1, 
                    validation_split=0.1)

Epoch 1/5
166/166 [==============================] - 23s 126ms/step - loss: 0.4878 - accuracy: 0.7917 - val_loss: 0.4466 - val_accuracy: 0.7959
Epoch 2/5
166/166 [==============================] - 19s 113ms/step - loss: 0.3104 - accuracy: 0.8675 - val_loss: 0.4471 - val_accuracy: 0.8197
Epoch 3/5
166/166 [==============================] - 22s 132ms/step - loss: 0.2366 - accuracy: 0.8994 - val_loss: 0.4858 - val_accuracy: 0.8180
Epoch 4/5
166/166 [==============================] - 24s 147ms/step - loss: 0.1794 - accuracy: 0.9285 - val_loss: 0.5895 - val_accuracy: 0.7891
Epoch 5/5
166/166 [==============================] - 21s 125ms/step - loss: 0.1361 - accuracy: 0.9459 - val_loss: 0.7503 - val_accuracy: 0.7993


In [54]:
prediction = model.predict(test_padded)
# Get labels based on probability 1 if p>= 0.5 else 0
pred_labels = []
for i in prediction:
    if i >= 0.5:
        pred_labels.append(1)
    else:
        pred_labels.append(0)
print("Accuracy of prediction on test set : ", accuracy_score(test_labels,pred_labels))

62/62 [==============================] - 2s 29ms/step
Accuracy of prediction on test set :  0.8096938775510204


# Based on the model, check the sentiment for the following two sentences
a. 'He is a great leader.'
b. 'He is a terrible leader.'

In [56]:
sentence = ['He is a great leader','He is a terrible leader.' ]
# convert to a sequence
sequences = tokenizer.texts_to_sequences(sentence)
# pad the sequence
padded = pad_sequences(sequences, padding='post', maxlen=max_length)
# Get labels based on probability 1 if p>= 0.5 else 0
prediction = model.predict(padded)
pred_labels = []


1/1 [==============================] - 0s 31ms/step
He is a great leader
Predicted sentiment :  Positive
He is a terrible leader.
Predicted sentiment :  Negative


In [57]:
for i in prediction:
    if i >= 0.5:
        pred_labels.append(1)
    else:
        pred_labels.append(0)
for i in range(len(sentence)):
    print(sentence[i])
    if pred_labels[i] == 1:
        s = 'Positive'
    else:
        s = 'Negative'
    print("Predicted sentiment : ",s)

He is a great leader
Predicted sentiment :  Positive
He is a terrible leader.
Predicted sentiment :  Negative


# Hence the prediction is good from the above sentiments